In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.utils import shuffle
import sklearn 
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm
import random
import warnings
warnings.filterwarnings('ignore')




In [31]:
# Total = np.unique(pd.read_csv("./188_wheeze.txt", header=None)[0].values)
Total = np.unique(breath_dataframe.iloc[:,73].values)
name = []
for i in Total:
    name.append(i[:4])
unique = []
before_after = []
for i in Total:
    if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
        unique.append(i)
    else:
        before_after.append(i) 

Asthmatic_Female = []
Asthmatic_Male = []
Healthy_Male = []
Healthy_Female = []
NAsthma = []
for file in unique:
    if file.split("_")[2] == 'Asthma' or file.split("_")[2] == 'asthma' :
        if file.find("_M_")!=-1:
            Asthmatic_Male.append(file)
        if file.find("_F_")!=-1:
            Asthmatic_Female.append(file)
    if file.split("_")[2] == 'NAsthma':
        NAsthma.append(file)
    if file.find("_C_")!=-1:
        if file.find("_M_")!=-1:
            Healthy_Male.append(file)
        if file.find("_F_")!=-1:
            Healthy_Female.append(file)

In [32]:
print(f"Total = {len(Total)}")
print(f"Asthmatic_male = {len(Asthmatic_Male)}")
print(f"Asthmatic_female = {len(Asthmatic_Female)}")
print(f"Healthy_Male = {len(Healthy_Male)}")
print(f"Healthy_female = {len(Healthy_Female)}")
print(f"Before_after = {len(before_after)}")
print(f"NAsthma = {len(NAsthma)}")

Total = 275
Asthmatic_male = 46
Asthmatic_female = 28
Healthy_Male = 55
Healthy_female = 51
Before_after = 84
NAsthma = 10


In [22]:
#Machine Learning Algorithm (MLA) Selection and Initialization
def MLA_selection(sound_dataframe, sound, path):    
    MLA = [
        #Ensemble Methods
#         ensemble.AdaBoostClassifier(),
#         ensemble.BaggingClassifier(),
#         ensemble.ExtraTreesClassifier(),
#         ensemble.GradientBoostingClassifier(),
        ensemble.RandomForestClassifier(),

        #Gaussian Processes
#         gaussian_process.GaussianProcessClassifier(),

        #GLM
#         linear_model.LogisticRegressionCV(),
#         linear_model.PassiveAggressiveClassifier(),
#         linear_model.RidgeClassifierCV(),
#         linear_model.SGDClassifier(),
#         linear_model.Perceptron(),

        #Navies Bayes
#         naive_bayes.BernoulliNB(),
#         naive_bayes.GaussianNB(),

        #Nearest Neighbor
#         neighbors.KNeighborsClassifier(),

        #SVM
        svm.SVC(probability=True),
#         svm.NuSVC(probability=True),
        svm.LinearSVC(),

        #Trees    
#         tree.DecisionTreeClassifier(),
# #         tree.ExtraTreeClassifier(),

        #Discriminant Analysis
#         discriminant_analysis.LinearDiscriminantAnalysis(),
#         discriminant_analysis.QuadraticDiscriminantAnalysis(),


        XGBClassifier(max_depth = 8,
                     subsample = 0.8,
                     learning_rate = 0.01,
                     n_estimators = 450,
                     min_child_weight = 1)
#         XGBClassifier()
        ]


    MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy','MLA Train Accuracy Mean', 'MLA Test Accuracy', 'MLA Test Accuracy Mean','MLA Test Accuracy Std' ]
    MLA_compare = pd.DataFrame(columns = MLA_columns)
    row_index = 0
    for alg in MLA:
        MLA_name = alg.__class__.__name__
        MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
        MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
        test = []
        train = []
        for i in tqdm(range(0,10)):
            train_csv, test_csv = pd.read_csv(path+'/train'+sound+str(i)+'.csv'), pd.read_csv(path+'./test'+sound+str(i)+'.csv')
            X_train = train_csv.iloc[:,:sound_dataframe.shape[1]-2]
            y_train = train_csv.iloc[:,sound_dataframe.shape[1]-2]
            X_test = test_csv.iloc[:,:sound_dataframe.shape[1]-2]
            y_test = test_csv.iloc[:,sound_dataframe.shape[1]-2]
            #Scaling
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            X_train = pd.DataFrame(X_train)
            X_test = pd.DataFrame(X_test)

            alg.fit(X_train,y_train)
            test.append(metrics.accuracy_score(y_test,alg.predict(X_test)))

            train.append(metrics.accuracy_score(y_train,alg.predict(X_train)))
            
#             if alg.__class__.__name__ == 'XGBClassifier':
#                 filename = './CMN/XGB_model'+ str(i) + '.model'
#                 pickle.dump(alg, open(filename, 'wb'))
#                 train_csv.to_csv('./CMN/Train_XGB'+str(i)+'.csv', index=False)
                
        
        MLA_compare.loc[row_index, 'MLA Train Accuracy'] = train
        MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = np.mean(train)
        MLA_compare.loc[row_index, 'MLA Test Accuracy'] = test
        MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = np.mean(test)
        MLA_compare.loc[row_index, 'MLA Test Accuracy Std'] = np.std(test)


        row_index+=1

    
    MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
    
    return MLA_compare

In [5]:
def make_legit_sets(sound_dataframe, sound, path):
    for i in range(0,10):
        Train = np.loadtxt(f"./{path}/Train/Train_{i}.txt", dtype= 'str')
        Test = np.loadtxt(f"./{path}/Test/Test_{i}.txt", dtype= 'str')


        test_dataframe = pd.DataFrame()
        for j, line in enumerate(Test):
            A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Test[j]))]
            test_dataframe = pd.DataFrame.append(test_dataframe,A)

        train_dataframe = pd.DataFrame()
        for j, line in enumerate(Train):
            A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Train[j]))]
            train_dataframe = pd.DataFrame.append(train_dataframe,A)
#################################################
        train_dataframe.to_csv('./breathe_set_csv_exp_5/train'+sound+str(i)+'.csv', index=False)
        test_dataframe.to_csv('./breathe_set_csv_exp_5/test'+sound+str(i)+'.csv', index=False)
    return None

In [6]:
Wheeze_dataframe = pd.read_csv("./breath_csv/Wheeze.csv")
Wheeze_dataframe_1 = pd.read_csv("./breath_csv/Wheeze_1.csv")
Wheeze_dataframe_2 = pd.read_csv("./breath_csv/Wheeze_2.csv")
Wheeze_dataframe_3 = pd.read_csv("./breath_csv/Wheeze_3.csv")

In [9]:
make_legit_sets(Wheeze_dataframe, 'Wheeze', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_1, 'Wheeze_1', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_2, 'Wheeze_2', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_3, 'Wheeze_3', "Wheeze_sets")

In [75]:
MLA_Wheeze = MLA_selection(Wheeze_dataframe, 'Wheeze', "./breathe_set_csv_exp_4/")
MLA_Wheeze_1 = MLA_selection(Wheeze_dataframe_1, 'Wheeze_1', "./breathe_set_csv_exp_4/")
MLA_Wheeze_2 = MLA_selection(Wheeze_dataframe_2, 'Wheeze_2', "./breathe_set_csv_exp_4/")
MLA_Wheeze_3 = MLA_selection(Wheeze_dataframe_3, 'Wheeze_3', "./breathe_set_csv_exp_4/")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[06:19:37] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:19:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:19:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:19:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[06:20:53] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:20:53] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:20:54] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:20:54] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [76]:
MLA_Wheeze

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9061224489795918, 0.8445378151260504, 0.890...",0.862794,0.0401354
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9142857142857143, 0.8697478991596639, 0.898...",0.857496,0.0493695
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9721577726218097, 0.97, 0.9664586583463338,...",0.972642,"[0.8979591836734694, 0.8151260504201681, 0.843...",0.815662,0.0482381
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8894044856921887, 0.9023076923076923, 0.883...",0.898692,"[0.8938775510204081, 0.7521008403361344, 0.839...",0.807796,0.0517178


In [77]:
MLA_Wheeze_1

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8571428571428571, 0.7647058823529411, 0.828...",0.784965,0.045702
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8285714285714286, 0.7815126050420168, 0.824...",0.78456,0.0395398
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8507347254447022, 0.8584615384615385, 0.851...",0.861377,"[0.8244897959183674, 0.7521008403361344, 0.859...",0.782321,0.0444023
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9303944315545244, 0.9276923076923077, 0.926...",0.932318,"[0.8081632653061225, 0.7563025210084033, 0.816...",0.779208,0.0401786


In [78]:
MLA_Wheeze_2

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9346938775510204, 0.8361344537815126, 0.875...",0.851052,0.0435679
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9346938775510204, 0.8529411764705882, 0.875...",0.845398,0.0509733
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9621036349574633, 0.9607692307692308, 0.961...",0.966538,"[0.926530612244898, 0.8529411764705882, 0.8164...",0.837902,0.0519563
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.888631090487239, 0.8923076923076924, 0.8923...",0.891199,"[0.889795918367347, 0.7605042016806722, 0.8125...",0.806633,0.0477326


In [79]:
MLA_Wheeze_3

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8646558391337974, 0.8784615384615385, 0.867...",0.872846,"[0.8857142857142857, 0.8025210084033614, 0.828...",0.803741,0.0384212
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9450889404485692, 0.946923076923077, 0.9446...",0.946774,"[0.889795918367347, 0.7983193277310925, 0.7812...",0.798646,0.0367506
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8734693877551021, 0.7773109243697479, 0.824...",0.794951,0.0451413
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8408163265306122, 0.773109243697479, 0.8320...",0.792802,0.0358028


In [60]:
Wheeze_dataframe_0 = pd.read_csv("./breath_csv/Wheeze_0.csv")
Wheeze_dataframe_5 = pd.read_csv("./breath_csv/Wheeze_5.csv")
Wheeze_dataframe_10 = pd.read_csv("./breath_csv/Wheeze_10.csv")
Wheeze_dataframe_15 = pd.read_csv("./breath_csv/Wheeze_15.csv")
Wheeze_dataframe_20 = pd.read_csv("./breath_csv/Wheeze_20.csv")
Wheeze_dataframe_25 = pd.read_csv("./breath_csv/Wheeze_25.csv")
Wheeze_dataframe_30 = pd.read_csv("./breath_csv/Wheeze_30.csv")
Wheeze_dataframe_35 = pd.read_csv("./breath_csv/Wheeze_35.csv")
Wheeze_dataframe_40 = pd.read_csv("./breath_csv/Wheeze_40.csv")
Wheeze_dataframe_45 = pd.read_csv("./breath_csv/Wheeze_45.csv")
Wheeze_dataframe_50 = pd.read_csv("./breath_csv/Wheeze_50.csv")

In [61]:
make_legit_sets(Wheeze_dataframe_0, 'Wheeze_0', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_5, 'Wheeze_5', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_10, 'Wheeze_10', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_15, 'Wheeze_15', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_20, 'Wheeze_20', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_25, 'Wheeze_25', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_30, 'Wheeze_30', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_35, 'Wheeze_35', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_40, 'Wheeze_40', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_45, 'Wheeze_45', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_50, 'Wheeze_50', "Wheeze_sets")

In [62]:
MLA_wheeze_0 = MLA_selection(Wheeze_dataframe_0, 'Wheeze_0', "./breathe_set_csv_exp_4/")
MLA_wheeze_5 = MLA_selection(Wheeze_dataframe_5, 'Wheeze_5', "./breathe_set_csv_exp_4/")
MLA_wheeze_10 = MLA_selection(Wheeze_dataframe_10, 'Wheeze_10', "./breathe_set_csv_exp_4/")
MLA_wheeze_15 = MLA_selection(Wheeze_dataframe_15, 'Wheeze_15', "./breathe_set_csv_exp_4/")
MLA_wheeze_20 = MLA_selection(Wheeze_dataframe_20, 'Wheeze_20', "./breathe_set_csv_exp_4/")
MLA_wheeze_25 = MLA_selection(Wheeze_dataframe_25, 'Wheeze_25', "./breathe_set_csv_exp_4/")
MLA_wheeze_30 = MLA_selection(Wheeze_dataframe_30, 'Wheeze_30', "./breathe_set_csv_exp_4/")
MLA_wheeze_35 = MLA_selection(Wheeze_dataframe_35, 'Wheeze_35', "./breathe_set_csv_exp_4/")
MLA_wheeze_40 = MLA_selection(Wheeze_dataframe_40, 'Wheeze_40', "./breathe_set_csv_exp_4/")
MLA_wheeze_45 = MLA_selection(Wheeze_dataframe_45, 'Wheeze_45', "./breathe_set_csv_exp_4/")
MLA_wheeze_50 = MLA_selection(Wheeze_dataframe_50, 'Wheeze_50', "./breathe_set_csv_exp_4/")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[05:50:03] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:50:08] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:50:18] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:50:23] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[05:58:55] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:58:58] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:59:02] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:59:05] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [63]:
MLA_wheeze_0

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8979591836734694, 0.8067226890756303, 0.878...",0.825883,0.0498517
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8979591836734694, 0.7941176470588235, 0.867...",0.824202,0.0438369
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9450889404485692, 0.9461538461538461, 0.944...",0.95146,"[0.8857142857142857, 0.8025210084033614, 0.820...",0.811478,0.0490403
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.877030162412993, 0.8838461538461538, 0.8728...",0.885553,"[0.9061224489795918, 0.7773109243697479, 0.859...",0.797247,0.0590706


In [64]:
MLA_wheeze_5

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9061224489795918, 0.7899159663865546, 0.851...",0.8227,0.052671
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9061224489795918, 0.8025210084033614, 0.859...",0.822147,0.0534777
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9466357308584686, 0.9484615384615385, 0.948...",0.951981,"[0.8816326530612245, 0.8277310924369747, 0.824...",0.811212,0.0552687
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8847641144624904, 0.89, 0.8806552262090483,...",0.893351,"[0.9102040816326531, 0.8109243697478992, 0.851...",0.808739,0.0588626


In [65]:
MLA_wheeze_10

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 0.9992125984251968, 1.0, 1.0, ...",0.999845,"[0.9183673469387755, 0.8067226890756303, 0.847...",0.826877,0.0522997
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.945862335653519, 0.9476923076923077, 0.9492...",0.951909,"[0.9061224489795918, 0.8613445378151261, 0.859...",0.825491,0.0520221
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9224489795918367, 0.7983193277310925, 0.875...",0.82279,0.0623617
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8863109048723898, 0.8838461538461538, 0.876...",0.888271,"[0.8857142857142857, 0.7352941176470589, 0.859...",0.79629,0.0592915


In [66]:
MLA_wheeze_15

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 0.9992125984251968, 1.0, 1.0, ...",0.999921,"[0.9346938775510204, 0.8319327731092437, 0.875...",0.842104,0.0498777
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9224489795918367, 0.8151260504201681, 0.855...",0.838738,0.0476872
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9559164733178654, 0.95, 0.9492979719188768,...",0.957191,"[0.9061224489795918, 0.8613445378151261, 0.859...",0.824198,0.0567637
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8847641144624904, 0.8923076923076924, 0.882...",0.892286,"[0.889795918367347, 0.7563025210084033, 0.8398...",0.808507,0.0559755


In [67]:
MLA_wheeze_20

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 0.9992125984251968, 1.0, 1.0, ...",0.999921,"[0.9306122448979591, 0.8487394957983193, 0.875...",0.845931,0.0498238
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9306122448979591, 0.8529411764705882, 0.875...",0.845395,0.0520998
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9551430781129157, 0.9507692307692308, 0.956...",0.960132,"[0.9224489795918367, 0.8697478991596639, 0.863...",0.834722,0.0555252
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8824439288476411, 0.8953846153846153, 0.884...",0.889604,"[0.9061224489795918, 0.7647058823529411, 0.843...",0.817977,0.0452526


In [68]:
MLA_wheeze_25

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9306122448979591, 0.8403361344537815, 0.867...",0.844822,0.0493536
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9346938775510204, 0.8445378151260504, 0.859...",0.842665,0.0478151
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9621036349574633, 0.9607692307692308, 0.961...",0.966538,"[0.926530612244898, 0.8529411764705882, 0.8164...",0.837902,0.0519563
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8870843000773395, 0.8915384615384615, 0.892...",0.891205,"[0.8857142857142857, 0.7563025210084033, 0.812...",0.804974,0.0477323


In [69]:
MLA_wheeze_30

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9387755102040817, 0.8445378151260504, 0.835...",0.84747,0.0465912
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9183673469387755, 0.819327731092437, 0.8476...",0.832815,0.0461284
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9597834493426141, 0.96, 0.9648985959438378,...",0.96709,"[0.926530612244898, 0.8151260504201681, 0.8398...",0.829502,0.047383
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8870843000773395, 0.8923076923076924, 0.886...",0.893852,"[0.9020408163265307, 0.7815126050420168, 0.789...",0.819318,0.0364086


In [70]:
MLA_wheeze_35

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9061224489795918, 0.8487394957983193, 0.820...",0.825188,0.0448311
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9597834493426141, 0.9646153846153847, 0.965...",0.965704,"[0.9061224489795918, 0.8277310924369747, 0.824...",0.823919,0.0418663
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9142857142857143, 0.8277310924369747, 0.816...",0.821434,0.0490248
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.888631090487239, 0.8869230769230769, 0.8822...",0.889634,"[0.8857142857142857, 0.7815126050420168, 0.777...",0.805244,0.0354808


In [71]:
MLA_wheeze_40

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9020408163265307, 0.8571428571428571, 0.820...",0.821839,0.0467198
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9613302397525135, 0.9607692307692308, 0.957...",0.963138,"[0.9020408163265307, 0.8151260504201681, 0.820...",0.817164,0.044771
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9992266047950503, 1.0, 0.999219968798752, 1...",0.999615,"[0.8979591836734694, 0.8403361344537815, 0.816...",0.814955,0.0507342
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8747099767981439, 0.8838461538461538, 0.875...",0.886102,"[0.8816326530612245, 0.7815126050420168, 0.761...",0.786197,0.0406657


In [72]:
MLA_wheeze_45

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9020408163265307, 0.8361344537815126, 0.808...",0.819029,0.040269
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 0.999219968798752, 0.99921259842519...",0.999766,"[0.8816326530612245, 0.8361344537815126, 0.820...",0.805194,0.0512598
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8669760247486465, 0.8738461538461538, 0.870...",0.876507,"[0.889795918367347, 0.773109243697479, 0.76171...",0.794319,0.041029
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9566898685228151, 0.9592307692307692, 0.955...",0.962747,"[0.889795918367347, 0.7941176470588235, 0.8046...",0.792668,0.0481288


In [73]:
MLA_wheeze_50

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8734693877551021, 0.8319327731092437, 0.785...",0.80965,0.0375153
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9992307692307693, 0.999219968798752, 0...",0.999538,"[0.889795918367347, 0.8277310924369747, 0.8320...",0.804897,0.0535399
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8739365815931941, 0.8853846153846154, 0.872...",0.882987,"[0.8734693877551021, 0.7563025210084033, 0.777...",0.797445,0.0407303
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9566898685228151, 0.96, 0.9609984399375975,...",0.963401,"[0.8734693877551021, 0.8109243697478992, 0.796...",0.79547,0.0406523


In [6]:
breath_dataframe = pd.read_csv("./breath_csv_2//Breath.csv")
breath_dataframe_1 = pd.read_csv("./breath_csv_2//Breath_1.csv")
breath_dataframe_2 = pd.read_csv("./breath_csv_2//Breath_2.csv")
breath_dataframe_3 = pd.read_csv("./breath_csv_2//Breath_3.csv")

In [16]:
make_legit_sets(breath_dataframe, 'breath', "breath_sets")
make_legit_sets(breath_dataframe_1, 'breath_1', "breath_sets")
make_legit_sets(breath_dataframe_2, 'breath_2', "breath_sets")
make_legit_sets(breath_dataframe_3, 'breath_3', "breath_sets")

In [23]:
MLA_breath = MLA_selection(breath_dataframe, 'breath', "./breathe_set_csv_exp_5/")
MLA_breath_1 = MLA_selection(breath_dataframe_1, 'breath_1', "./breathe_set_csv_exp_5/")
MLA_breath_2 = MLA_selection(breath_dataframe_2, 'breath_2', "./breathe_set_csv_exp_5/")
MLA_breath_3 = MLA_selection(breath_dataframe_3, 'breath_3', "./breathe_set_csv_exp_5/")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[13:49:13] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:49:19] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:49:24] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:49:31] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[13:54:24] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:54:30] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:54:36] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:54:41] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [18]:
MLA_breath

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7868217054263565, 0.6887966804979253, 0.833...",0.752402,0.0624844
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8449612403100775, 0.7012448132780082, 0.829...",0.749282,0.0696579
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8207954000958313, 0.8393536121673004, 0.838...",0.840507,"[0.7984496124031008, 0.7012448132780082, 0.755...",0.725006,0.0537489
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9573550551030187, 0.9591254752851711, 0.955...",0.958557,"[0.7713178294573644, 0.6597510373443983, 0.748...",0.721273,0.066679


In [19]:
MLA_breath_1

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7906976744186046, 0.6804979253112033, 0.802...",0.72422,0.0479949
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9118351701006229, 0.9115969581749049, 0.906...",0.91234,"[0.7713178294573644, 0.6929460580912863, 0.775...",0.723674,0.0505467
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7868217054263565, 0.6804979253112033, 0.779...",0.71679,0.0567339
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.7834211787254433, 0.7984790874524715, 0.791...",0.791539,"[0.7441860465116279, 0.6390041493775933, 0.717...",0.690223,0.0403104


In [20]:
MLA_breath_2

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7751937984496124, 0.7012448132780082, 0.794...",0.729605,0.0694217
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7713178294573644, 0.6929460580912863, 0.771...",0.726234,0.0601997
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8241494968854816, 0.8365019011406845, 0.830...",0.835535,"[0.7829457364341085, 0.6970954356846473, 0.713...",0.702954,0.0574302
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9405845711547676, 0.9434410646387833, 0.938...",0.941855,"[0.7286821705426356, 0.6473029045643154, 0.705...",0.697697,0.0643201


In [21]:
MLA_breath_3

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8207954000958313, 0.8388783269961977, 0.829...",0.83122,"[0.8488372093023255, 0.7136929460580913, 0.767...",0.763883,0.0457585
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9175850503114519, 0.9206273764258555, 0.921...",0.922433,"[0.8023255813953488, 0.6804979253112033, 0.732...",0.74148,0.0546907
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8178294573643411, 0.7012448132780082, 0.779...",0.737633,0.0543026
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.813953488372093, 0.6804979253112033, 0.7596...",0.735487,0.0573737


In [24]:
MLA_breath

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8372093023255814, 0.6929460580912863, 0.837...",0.758418,0.0660955
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8178294573643411, 0.7053941908713693, 0.829...",0.753876,0.0615511
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8203162434115956, 0.8393536121673004, 0.838...",0.840175,"[0.7984496124031008, 0.7012448132780082, 0.755...",0.725388,0.0530934
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9573550551030187, 0.9591254752851711, 0.955...",0.958557,"[0.7713178294573644, 0.6597510373443983, 0.748...",0.721273,0.066679


In [25]:
MLA_breath_1

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7945736434108527, 0.6929460580912863, 0.786...",0.724995,0.0413475
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9118351701006229, 0.9115969581749049, 0.906...",0.91234,"[0.7713178294573644, 0.6929460580912863, 0.775...",0.723674,0.0505467
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9990416866315285, 0.9990494296577946, 1.0, ...",0.999186,"[0.7790697674418605, 0.6887966804979253, 0.806...",0.720204,0.0481873
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.7834211787254433, 0.7994296577946768, 0.790...",0.791682,"[0.7403100775193798, 0.6514522821576764, 0.709...",0.688689,0.0389528


In [26]:
MLA_breath_2

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995208433157643, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.999952,"[0.7829457364341085, 0.6763485477178424, 0.821...",0.736441,0.0651527
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7403100775193798, 0.6929460580912863, 0.802...",0.725029,0.058946
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8251078102539531, 0.8350760456273765, 0.831...",0.83501,"[0.7829457364341085, 0.6970954356846473, 0.709...",0.702585,0.0573989
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9405845711547676, 0.9434410646387833, 0.938...",0.941855,"[0.7286821705426356, 0.6473029045643154, 0.705...",0.697697,0.0643201


In [27]:
MLA_breath_3

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8212745567800671, 0.839828897338403, 0.8298...",0.831176,"[0.8372093023255814, 0.7136929460580913, 0.767...",0.762629,0.0429664
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9175850503114519, 0.9206273764258555, 0.921...",0.922433,"[0.8023255813953488, 0.6804979253112033, 0.732...",0.74148,0.0546907
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9995247148288974, 0.9995208433157643, ...",0.99957,"[0.8255813953488372, 0.6929460580912863, 0.779...",0.741081,0.0562128
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8178294573643411, 0.7095435684647303, 0.786...",0.737031,0.063709


array(['4001_C_Na_M_Na_Na_Y.json', '4004_P_Asthma_m_54_Y.json',
       '4011_P_Asthma_F_after_TBD_Y.json',
       '4027_P_Asthma_F_after_Mod_Y.json',
       '4027_P_Asthma_F_before_Mod_Y.json', '4032_C_Na_M_Na_Na_N.json',
       '4033_C_Na_M_Na_Na_N.json', '4038_C_Na_F_Na_Na_Y.json',
       '4040_C_Na_F_Na_Na_Y.json', '4043_C_Na_F_Na_Na_N.json',
       '4054_P_Asthma_F_after_Mod_Y.json',
       '4054_P_Asthma_F_before_Mod_Y.json',
       '4060_P_Asthma_F_after_TBD_N.json', '4064_C_Na_M_Na_Na_Y.json',
       '4065_P_Asthma_M_after_Sev_Y.json',
       '4065_P_Asthma_M_before_Sev_Y.json',
       '4067_P_Asthma_M_before_Mil_Y.json', '4068_C_Na_F_Na_Na_Y.json',
       '4069_C_Na_M_Na_Na_Y.json', '4070_C_Na_F_Na_Na_Y.json',
       '4071_C_Na_M_Na_Na_Y.json', '4074_C_Na_F_Na_Na_N.json',
       '4076_C_Na_M_Na_Na_N.json', '4082_P_Asthma_M_before_Mod_Y.json',
       '4083_C_Na_M_Na_Na_Y.json', '4084_P_NAsthma_F_before_TBD_Y.json',
       '4089_P_Asthma_F_after_Mod_Y.json',
       '4089_P_Asthma